In [1]:
# Out of date!
from wit import Wit
import pandas as pd

import re

ModuleNotFoundError: No module named 'wit'

In [ ]:
def loadData():
    return pd.read_csv("https://raw.githubusercontent.com/KartikChugh/Otto/master/src/static/datasets/tweets.csv", encoding="latin-1")

In [ ]:
FEATURE_COLUMN = 1
DATA_COUNT = 100

In [ ]:
data = loadData()

In [ ]:
# Selects column to analyze
data = data.iloc[:, FEATURE_COLUMN]
# Keeps specified number of rows
data = data.sample(n=DATA_COUNT, random_state=42)

In [ ]:
data = data.str.strip()
data = data.replace('\?|\.|\!|\/|\;|\:|\#', '', regex=True)

In [ ]:
client = Wit("GUNUX2F3ZIXVVLK3DCRY6NXH2OLGOG6I")

In [ ]:
# Reformats trait/entity names
def rename(name):
    name = name[name.find(':')+1:]
    name = name[name.find('$')+1:]
    name = name.title()
    name = name.replace("Sentiment", "Polarity")
    return name

In [ ]:
# Converts wit's response into readable form
def traitLabelForSample(traits):
    traitArr = []
    for traitName in traits:
        matches = traits[traitName]
        firstMatch = matches[0]
        value = firstMatch['value']
        traitArr.append(rename(traitName) + ": " + value)
    traitLabel = ', '.join(traitArr)
    return traitLabel

In [ ]:
# Converts wit's response into readable form
def entityLabelForSample(entities):
    entityArr = []
    for entityName in entities:
        matches = entities[entityName]
        firstMatch = matches[0]
        value = firstMatch['body']
        entityArr.append(rename(entityName) + ": " + value)
    entityLabel = ', '.join(entityArr)
    return entityLabel

In [ ]:
# Data
texts = []
entityLabels = []
traitLabels = []

# Queries Wit for NLP analysis, and processes/stores result
for i, row in data.iteritems():
    text = str(row)
    resp = client.message(text)

    entities = resp.get('entities')
    traits = resp.get('traits')
    entityLabel = entityLabelForSample(entities)
    traitLabel = traitLabelForSample(traits)

    entityLabels.append(entityLabel)
    traitLabels.append(traitLabel)
    texts.append(text)

In [ ]:
labeled = pd.DataFrame({'Text':texts,'Sentiments':traitLabels, 'Entities':entityLabels})

In [ ]:
labeled.head(DATA_COUNT)